In [1]:
!pip install transformers
!pip install wget

     |████████████████████████████████| 2.1MB 7.8MB/s 
     |████████████████████████████████| 901kB 37.8MB/s 
     |████████████████████████████████| 3.3MB 54.6MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=693f50523f02fed70bacc72856fdce3583c3e74012b527a9552fe782e84e7df9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [22]:
# Datenvorbereitung
import json
import wget

# download and open json file
url = 'https://raw.githubusercontent.com/Johannes96/BERT_fineTune/master/data/final_train_asc_CLS.json'
dataset1 = wget.download(url)

with open(dataset1) as json_file:
    train_asc = json.load(json_file)

url = 'https://raw.githubusercontent.com/Johannes96/BERT_fineTune/master/data/final_dev_asc_CLS.json'
dataset2 = wget.download(url)

with open(dataset2) as json_file:
    dev_asc = json.load(json_file)

# Extrahiere sentences und labels aus den Daten
train_texts = []
train_labels_temp = []
for i in train_asc:
    train_labels_temp.append(train_asc[i]['polarity'])
    train_texts.append(train_asc[i]['sentence'])

dev_texts = []
dev_labels_temp = []
for i in dev_asc:
    dev_labels_temp.append(dev_asc[i]['polarity'])
    dev_texts.append(dev_asc[i]['sentence'])

# Ersetzte labels durch Nummern von 0 bis 4
train_labels = []
for i in train_labels_temp:
    if i == 'str_negative':
        train_labels.append(0)
    elif i == 'negative':
        train_labels.append(1)
    elif i == 'neutral':
        train_labels.append(2)
    elif i == 'positive':
        train_labels.append(3)
    elif i == 'str_positive':
        train_labels.append(4)

dev_labels = []
for i in dev_labels_temp:
    if i == 'str_negative':
        dev_labels.append(0)
    elif i == 'negative':
        dev_labels.append(1)
    elif i == 'neutral':
        dev_labels.append(2)
    elif i == 'positive':
        dev_labels.append(3)
    elif i == 'str_positive':
        dev_labels.append(4)

# Verreinige Listen zu einem Datensatz
texts = train_texts
labels = train_labels
labels.extend(dev_labels)
texts.extend(dev_texts)

# Teile in Trainings- und Testdaten auf
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=.2)


In [23]:
# Load Tokenizer
from transformers import DebertaTokenizer
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')

# Tokenizing der Trainings- und Testdaten
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
# test_encodings = tokenizer(dev_texts, truncation=True, padding=True)

# erstelle Datensätze
import torch

class ASCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ASCDataset(train_encodings, train_labels)
val_dataset = ASCDataset(val_encodings, val_labels)
# test_dataset = ASCDataset(test_encodings, dev_labels)

# Anpassung von DeBERTa sodass er mit 5 labels funktioniert
from transformers import DebertaForSequenceClassification, DebertaConfig, Trainer, TrainingArguments
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base')

config = DebertaConfig.from_pretrained('microsoft/deberta-base')
config.num_labels = 5 # adjust to change number of distinct labels (y)
model = DebertaForSequenceClassification(config)
# print(model.parameters) # show how many labels (out_features = ___) are defined

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.weight

In [24]:
# Trainingsphase
trainer.train()

Step,Training Loss
10,1.565700
20,1.499200
30,1.471300
40,1.470500
50,1.450900
60,1.426100
70,1.464600
80,1.418900
90,1.456900
100,1.429000


TrainOutput(global_step=1377, training_loss=1.1426834841958837, metrics={'train_runtime': 669.9972, 'train_samples_per_second': 2.055, 'total_flos': 1948606557050340.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 557564416, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -453046272, 'train_mem_gpu_alloc_delta': 1684273152, 'train_mem_cpu_peaked_delta': 457011200, 'train_mem_gpu_peaked_delta': 2418739712})

In [25]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_loss': 1.0642540454864502,
 'eval_mem_cpu_alloc_delta': 176128,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 593602560,
 'eval_runtime': 12.8466,
 'eval_samples_per_second': 142.839}

In [26]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-78904017-5fe0-86b9-a844-ac9247c966f3)
